# 🧾 View as a summary

In [ ]:
#| default_exp repr_str

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
# |hide
import os
from nbdev.showdoc import *
from fastcore.test import test_eq, test

In [ ]:
#| hide

os.environ["DEBUG"]="0"
os.environ["CPU"]="1"

In [ ]:
#| hide
#| export

import warnings
from typing import Union, Optional as O

import numpy as np
# import jax, jax.numpy as jnp

from lovely_numpy import np_to_str_common, pretty_str, sparse_join, ansi_color, in_debugger, bytes_to_human
from lovely_numpy import config as lnp_config

from lovely_grad.utils.config import get_config, config, set_config
from lovely_grad.utils.misc import is_cpu

import tinygrad.helpers, tinygrad.tensor
from tinygrad.tensor import Tensor, DType, dtypes

In [ ]:
# |hide
np.random.seed(1337)

randoms = np.random.randn(100,).astype(np.float32)

spicy = randoms[:12].copy()
spicy[0] *= 10000.0
spicy[1] /= 10000.0
spicy[3] = np.Inf
spicy[4] = np.NINF
spicy[5] = np.NaN
spicy = spicy.reshape((2,6))


# Works with gpu too, but I keep cpu for CI testing to match the outputs.
randoms = Tensor(randoms, device="cpu")
spicy = Tensor(spicy, device="cpu")

In [ ]:
# |exporti
dtnames =   {   "half": "f16",
                "float": "f32",
                "char": "i8",
                "uchar": "u8",
                "int":   "i32",
                "int64": "i64",
            }


def short_dtype(x: DType) -> str:
    return dtnames.get(x.dtype.name, str(x.dtype)) if x.dtype != Tensor.default_type else ""

In [ ]:
# |hide
# test_eq(short_dtype(jnp.array(1., dtype=jnp.bfloat16)), "bf16")

In [ ]:
# | exporti
def plain_repr(x: Tensor):
    "Pick the right function to get a plain repr"
    # assert isinstance(x, np.ndarray), f"expected np.ndarray but got {type(x)}" # Could be a sub-class.
    return x._plain_repr() if hasattr(x, "_plain_repr") else repr(x)

# def plain_str(x: torch.Tensor):
#     "Pick the right function to get a plain str."
#     # assert isinstance(x, np.ndarray), f"expected np.ndarray but got {type(x)}"
#     return x._plain_str() if hasattr(type(x), "_plain_str") else str(x)

In [ ]:
# | exporti
def is_nasty(x: Tensor):
    """Return true of any `x` values are inf or nan"""
    if x.shape == (): return False # min/max don't like zero-lenght arrays
    
    x_min = x.min().numpy().squeeze()
    x_max = x.max().numpy().squeeze()

    return np.isnan(x_min) or np.isinf(x_min) or np.isinf(x_max)

In [ ]:
#| hide
# test_eq(is_nasty(Tensor([1, 2, float("nan")])), True) ### Fix tinygrad/#862 first
test_eq(is_nasty(Tensor([1, 2, float("inf")])), True)
test_eq(is_nasty(Tensor([1, 2, 3])), False)
# test_eq(is_nasty(Tensor([])), False)

In [ ]:
# |export
def tensor_to_str_common(x: Tensor,  # Input
                        color=True,  # ANSI color highlighting
                        ddof=0):     # For "std" unbiasing

    if x.numel() == 0: return ansi_color("empty", "grey", color)
    if x.eq(0).min().eq(1).numpy(): return ansi_color("all_zeros", "grey", color)

    if x.ndim > 0:
        x_min = x.min().numpy().squeeze()
        x_max = x.max().numpy().squeeze()
        minmax = f"x∈[{pretty_str(x_min)}, {pretty_str(x_max)}]" if x.numel() > 2 else None

        # XXX Add bias correction?
        x_mean = x.mean().numpy().squeeze()
        x_std = x.std().numpy().squeeze()
        meanstd = f"μ={pretty_str(x_mean)} σ={pretty_str(x_std)}" if x.numel() >= 2 else None

        return sparse_join([minmax, meanstd])

In [ ]:
# |exporti

def to_str(x: Tensor,  # Input
            verbose:        bool    =False,
            auto_realize:    O[bool] =None,
            depth:          int     =0,
            lvl:            int     =0,
            color:          O[bool] =None
        ) -> str:

    # if plain:
    #     return plain_repr(x)

    conf = get_config()
    if color is None: color=conf.color
    if auto_realize is None: auto_realize=conf.auto_realize

    if in_debugger(): color = False


    tname = type(x).__name__.split(".")[-1]             # Tensor
    shape = str(list(x.shape)) if (x.ndim) else None   # [1,2,3]
    type_str = sparse_join([tname, shape], sep="")      # Tensor[1,2,3]

    dtype = short_dtype(x)                              # f16
    dev = x.device                                      # CPU

    grad = "grad" if x.requires_grad else None          # grad
    if x.grad is not None: grad = grad + ansi_color("+", "green", color)


    numel = None
    if x.shape and max(x.shape) != x.numel():
        numel = f"n={x.ndim}"
        # if get_config().show_mem_above <= x.nbytes:
        #     numel = sparse_join([numel, f"({bytes_to_human(x.nbytes)})"])
    # elif get_config().show_mem_above <= x.nbytes:
        # numel = bytes_to_human(x.nbytes)


    just_realized = None
    if auto_realize and not x.lazydata.realized:
        just_realized = ansi_color("Realized "+ str(x.lazydata.op.op).split(".")[-1], "grey", color)
        x.realize()

    res  = ""
    if x.lazydata.realized:
        # `lovely-numpy` is used to calculate stats when doing so on GPU would require
        # memory allocation (no-float tensors, tensors with bad numbers),
        #
        # Temporarily set the numpy config to match our config for consistency.
        with lnp_config(precision=conf.precision,
                        threshold_min=conf.threshold_min,
                        threshold_max=conf.threshold_max,
                        sci_mode=conf.sci_mode):

            if is_nasty(x) or not x.is_floating_point():
                common = np_to_str_common(x.numpy(), color=color)
            else:
                common = tensor_to_str_common(x, color=color)

            vals = pretty_str(x.numpy()) if 0 < x.numel() <= 10 else None
            res = sparse_join([type_str, dtype, numel, common, grad, dev,  vals, just_realized])
    else:
        op = "Lazy " + str(x.lazydata.op.op).split(".")[-1]
        res = sparse_join([type_str, dtype, numel, grad, dev, op])
    # else:
    #     res = plain_repr(x)

    if verbose:
        res += "\n" + plain_repr(x)

    if depth and x.ndim > 1:
        with config(show_mem_above=np.inf):
            deep_width = min((x.shape[0]), conf.deeper_width) # Print at most this many lines
            deep_lines = [ " "*conf.indent*(lvl+1) + to_str(x[i,:].realize(), depth=depth-1, lvl=lvl+1)
                                for i in range(deep_width)] 

            # If we were limited by width, print ...
            if deep_width < x.shape[0]: deep_lines.append(" "*conf.indent*(lvl+1) + "...")

            res += "\n" + "\n".join(deep_lines)

    return res

In [ ]:
# |exporti
def history_warning():
    "Issue a warning (once) ifw e are running in IPYthon with output cache enabled"

    if "get_ipython" in globals() and get_ipython().cache_size > 0:
        warnings.warn("IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html")

In [ ]:
# |hide
get_ipython().cache_size=1000
history_warning()

/tmp/ipykernel_87341/3648473780.py:6: UserWarning: IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html
  warnings.warn("IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html")


In [ ]:
# |hide
get_ipython().cache_size=0

In [ ]:
#| exporti

class StrProxy():
    def __init__(self, x: Tensor, plain=False, verbose=False, depth=0, lvl=0, color=None):
        self.x = x
        self.plain = plain
        self.verbose = verbose
        self.depth=depth
        self.lvl=lvl
        self.color=color
        history_warning()
    
    def __repr__(self):
        if self.plain: return plain_repr(self.x)
        return to_str(self.x, verbose=self.verbose,
                      depth=self.depth, lvl=self.lvl, color=self.color)

    # This is used for .deeper attribute and .deeper(depth=...).
    # The second onthe results in a __call__.
    def __call__(self, depth=1):
        return StrProxy(self.x, depth=depth)

In [ ]:
# |export
def lovely(x: Tensor, # Tensor of interest
            verbose=False,  # Whether to show the full tensor
            depth=0,        # Show stats in depth
            color=None):    # Force color (True/False) or auto.
    return StrProxy(x, verbose=verbose, depth=depth, color=color)

### Examples

##### Control laziness of repr

In [ ]:
set_config(auto_realize=False)
lovely(spicy),

(Tensor[2, 6] n=2 CPU Lazy FROMCPU,)

In [ ]:
lovely(spicy)

Tensor[2, 6] n=2 CPU Lazy FROMCPU

In [ ]:
set_config(auto_realize=True)
lovely(spicy)

Tensor[2, 6] n=2 x∈[-7.032e+03, 1.549] μ=-781.232 σ=2.210e+03 +Inf! -Inf! NaN! CPU Realized FROMCPU

In [ ]:
lovely(spicy)

Tensor[2, 6] n=2 x∈[-7.032e+03, 1.549] μ=-781.232 σ=2.210e+03 +Inf! -Inf! NaN! CPU

##### Show the stats and values

In [ ]:
lovely(randoms[0])

Tensor[1] CPU [-0.703] Realized SHRINK

In [ ]:
lovely(randoms[:2])

Tensor[2] μ=-0.597 σ=0.106 CPU [-0.703, -0.490] Realized SHRINK

In [ ]:

lovely(randoms[:6].reshape((2, 3))), # More than 2 elements -> show statistics

(Tensor[2, 3] n=2 x∈[-2.011, 0.207] μ=-0.846 σ=0.787 CPU [[-0.703, -0.490, -0.322], [-1.755, 0.207, -2.011]] Realized RESHAPE,)

In [ ]:
lovely(randoms[:11])                # More than 10 -> suppress data output

Tensor[11] x∈[-2.011, 1.549] μ=-0.336 σ=1.108 CPU Realized SHRINK

In [ ]:
# |hide
# test_eq(str(lovely(randoms[0])),                'Tensor[1] CPU \x1b[38;2;127;127;127mRealized SHRINK\x1b[0m [-0.703]')
# test_eq(str(lovely(randoms[:2])),               'Tensor[2] μ=-0.597 σ=0.106 CPU \x1b[38;2;127;127;127mRealized SHRINK\x1b[0m [-0.703, -0.490]')
# test_eq(str(lovely(randoms[:6].reshape(2, 3))), 'Tensor[2, 3] n=2 x∈[-2.011, 0.207] μ=-0.846 σ=0.787 CPU \x1b[38;2;127;127;127mRealized RESHAPE\x1b[0m [[-0.703, -0.490, -0.322], [-1.755, 0.207, -2.011]]')
# test_eq(str(lovely(randoms[:11])),              'Tensor[11] x∈[-2.011, 1.549] μ=-0.336 σ=1.108 CPU')

##### Gradient

In [ ]:
g=Tensor([1,2,3], requires_grad=True)
lovely(g)

Tensor[3] x∈[1.000, 3.000] μ=2.000 σ=0.816 grad CPU [1.000, 2.000, 3.000] Realized FROMCPU

In [ ]:
(g*g).sum().backward()
lovely(g)

Tensor[3] x∈[1.000, 3.000] μ=2.000 σ=0.816 grad+ CPU [1.000, 2.000, 3.000]

::: {.callout-note}

Note the green '<span style="color: green;">+</span>'  when the gradient is available.

:::

In [ ]:
lovely(g.grad)

Tensor[3] x∈[2.000, 6.000] μ=4.000 σ=1.633 CPU [2.000, 4.000, 6.000] Realized ADD

##### Do we have __any__ floating point nasties?

In [ ]:
# Statistics and range are calculated on good values only, if there are at lest 3 of them.
lovely(spicy)

Tensor[2, 6] n=2 x∈[-7.032e+03, 1.549] μ=-781.232 σ=2.210e+03 +Inf! -Inf! NaN! CPU

In [ ]:
lovely(spicy, color=False)

Tensor[2, 6] n=2 x∈[-7.032e+03, 1.549] μ=-781.232 σ=2.210e+03 +Inf! -Inf! NaN! CPU

In [ ]:
lovely(Tensor([float("nan")]*11))

Tensor[11] NaN! CPU Realized FROMCPU

##### Is the tensor __all__ zeros?

In [ ]:
lovely(Tensor.zeros(12))

Tensor[12] all_zeros CPU Realized CONTIGUOUS

In [ ]:
# |hide
# test_array_repr(str(lovely(jnp.zeros(12))),
#         'Array[12] \x1b[38;2;127;127;127mall_zeros\x1b[0m gpu:0')

In [ ]:
# XXX empty tensors - fix when they work
# lovely(jnp.array([], dtype=jnp.float16).reshape((0,0,0)))

In [ ]:
# |hide
# test_array_repr(str(lovely(jnp.array([], dtype=jnp.float16).reshape((0,0,0)))),
#         'Array[0, 0, 0] f16 \x1b[38;2;127;127;127mempty\x1b[0m gpu:0')

##### Shows the dtype if it's not the default.

In [ ]:
lovely(Tensor([1,2,3], dtype=dtypes.int8).realize())

Tensor[3] i8 x∈[1, 3] μ=2.000 σ=0.816 CPU [1, 2, 3]

In [ ]:
# |hide
# test_array_repr(str(lovely(jnp.array([1,2,3], dtype=jnp.int32))),
#         'Array[3] i32 x∈[1, 3] μ=2.000 σ=0.816 gpu:0 [1, 2, 3]')

In [ ]:
lovely(spicy, verbose=True)

Tensor[2, 6] n=2 x∈[-7.032e+03, 1.549] μ=-781.232 σ=2.210e+03 +Inf! -Inf! NaN! CPU
<Tensor buffer<12, dtypes.float> on CPU with grad None>

##### We need to go deeper

In [ ]:
image = np.load("mysteryman.npy")
image[1,2,3] = float('nan')

image = Tensor(image)

lovely(image, depth=2) # Limited by set_config(deeper_lines=N)

Tensor[3, 196, 196] n=3 x∈[-2.118, 2.640] μ=-0.388 σ=1.073 NaN! CPU Realized FROMCPU
  Tensor[196, 196] n=2 x∈[-2.118, 2.249] μ=-0.324 σ=1.036 CPU
    Tensor[196] x∈[-1.912, 2.249] μ=-0.673 σ=0.521 CPU
    Tensor[196] x∈[-1.861, 2.163] μ=-0.738 σ=0.417 CPU
    Tensor[196] x∈[-1.758, 2.198] μ=-0.806 σ=0.396 CPU
    Tensor[196] x∈[-1.656, 2.249] μ=-0.849 σ=0.368 CPU
    Tensor[196] x∈[-1.673, 2.198] μ=-0.857 σ=0.356 CPU
    Tensor[196] x∈[-1.656, 2.146] μ=-0.848 σ=0.371 CPU
    Tensor[196] x∈[-1.433, 2.215] μ=-0.784 σ=0.396 CPU
    Tensor[196] x∈[-1.279, 2.249] μ=-0.695 σ=0.485 CPU
    Tensor[196] x∈[-1.364, 2.249] μ=-0.637 σ=0.538 CPU
    ...
  Tensor[196, 196] n=2 x∈[-1.966, 2.429] μ=-0.274 σ=0.973 NaN! CPU
    Tensor[196] x∈[-1.861, 2.411] μ=-0.529 σ=0.555 CPU
    Tensor[196] x∈[-1.826, 2.359] μ=-0.562 σ=0.472 CPU
    Tensor[196] x∈[-1.756, 2.376] μ=-0.622 σ=0.458 NaN! CPU
    Tensor[196] x∈[-1.633, 2.429] μ=-0.664 σ=0.429 CPU
    Tensor[196] x∈[-1.651, 2.376] μ=-0.669 σ=0.398 CPU
   